# Wrangle Polity Data

### Input Datasets

- `states2016.csv`
- `Non-StateWarData_v4.0.csv`
- `INTRA-STATE_State_participants v5.1 CSV.csv`
- `Extra-StateWarData_v4.0.csv`

### Output Datasets

- `polity.csv`
- `polity_dates.csv`

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data_path = "../data/raw/"
processed_data_path = "../data/processed/"

In [3]:
dfStates = pd.read_csv(raw_data_path+'states2016.csv', encoding='utf-8')
dfNonStateWarEntities = pd.read_csv(raw_data_path+'Non-StateWarData_v4.0.csv', usecols=['SideA1', 'SideA2', 'SideB1', 'SideB2', 'SideB3', 'SideB4', 'SideB5'], encoding='utf-8', na_values=[-7, -8, -9])
dfIntraStateWarEntities = pd.read_csv(raw_data_path+'INTRA-STATE_State_participants v5.1 CSV.csv', usecols=['CcodeA', 'SideA', 'CcodeB', 'SideB'], encoding='latin-1', na_values=[-7, -8, -9])
dfExtraStateWarEntities = pd.read_csv(raw_data_path+'Extra-StateWarData_v4.0.csv', usecols=['ccode1', 'SideA', 'ccode2', 'SideB'], encoding='latin-1', na_values=[-7, -8, -9])

## Create "polity" table

Table creation statement:

```
class Polity(Base):
    __tablename__ = "polity"

    id = Column(Integer(5), primary_key=True)
    name = Column(Text)
    type = Column(Text)
    abbr = Column(String(3))
```

In [4]:
dfStates_POL = dfStates[['stateabb', 'ccode', 'statenme']].copy() \
    .drop_duplicates() \
    .rename(columns={'stateabb':'abbr', 'ccode':'id', 'statenme':'name'})

dfStates_POL['type'] = 'State'
dfStates_POL = dfStates_POL[['id', 'name', 'type', 'abbr']]
dfStates_POL

,id,name,type,abbr
0,2,United States of America,State,USA
1,20,Canada,State,CAN
2,31,Bahamas,State,BHM
3,40,Cuba,State,CUB
5,41,Haiti,State,HAI
...,...,...,...,...
238,970,Nauru,State,NAU
239,983,Marshall Islands,State,MSI
240,986,Palau,State,PAL
241,987,Federated States of Micronesia,State,FSM


In [5]:
IOrows = [(0, 'League of Nations', 'International Organization', 'LN'),
          (1, 'United Nations', 'International Organization', 'UN')]
dfIOrows = pd.DataFrame(IOrows, columns=['id', 'name', 'type', 'abbr'])

In [6]:
dfNonStateWarEntities

,SideA1,SideA2,SideB1,SideB2,SideB3,SideB4,SideB5
0,Te Rauparaha's Ngati Toa,NaN,Taranaki,Ngai Tahu,Waikato,Ngati Ira,Rangitikei
1,Shaka Zulu,NaN,Bantu,NaN,NaN,NaN,NaN
2,Burma,NaN,Assam,NaN,NaN,NaN,NaN
3,Buenos Aires,NaN,Provinces,NaN,NaN,NaN,NaN
4,Hongi Hika's Nga Phuhi,NaN,Ngati Paoa,Ngati Maru,Waikato River Maori,Te Arawa,NaN
...,...,...,...,...,...,...,...
57,Hutu,NaN,Tutsi,NaN,NaN,NaN,NaN
58,Dhofar,NaN,Oman,NaN,NaN,NaN,NaN
59,MPLA,NaN,FLNA,UNITA,NaN,NaN,NaN
60,Fretilin,Apodeti,UDT,NaN,NaN,NaN,NaN


In [7]:
dfNSW_POL = dfNonStateWarEntities.melt(value_vars=dfNonStateWarEntities.columns, value_name="name")\
    .drop(columns=["variable"]).dropna()
dfNSW_POL["name"] = dfNSW_POL["name"].str.strip()
dfNSW_POL = dfNSW_POL.drop_duplicates()
dfNSW_POL["id"] = np.NaN
dfNSW_POL

,name,id
0,Te Rauparaha's Ngati Toa,NaN
1,Shaka Zulu,NaN
2,Burma,NaN
3,Buenos Aires,NaN
4,Hongi Hika's Nga Phuhi,NaN
...,...,...
248,Waikato,NaN
252,Waikato River Maori,NaN
310,Ngati Ira,NaN
314,Te Arawa,NaN


In [8]:
dfIntraStateWarEntities

,CcodeA,SideA,CcodeB,SideB
0,365.0,Russia,NaN,Caucasus Rebels
1,300.0,Austria,NaN,NaN
2,329.0,Two Sicilies,NaN,Liberals
3,640.0,Ottoman Empire,NaN,Ali Pasha Loyalists
4,300.0,Austria,NaN,NaN
...,...,...,...,...
588,530.0,Ethiopia,NaN,NaN
589,NaN,NaN,365.0,Russia
590,369.0,Ukraine,NaN,Separatists
591,620.0,Libya,NaN,Libyan Dawn


In [9]:
dfISW_POL = pd.concat([dfIntraStateWarEntities[["CcodeA", "SideA"]]
                           .rename(columns={"CcodeA":"id", "SideA":"name"}), 
                       dfIntraStateWarEntities[["CcodeB", "SideB"]]
                           .rename(columns={"CcodeB":"id", "SideB":"name"})])\
                .dropna(how="all")
dfISW_POL = dfISW_POL[dfISW_POL["id"].isna()]
dfISW_POL

,id,name
10,NaN,Damascus and Aleppo
12,NaN,Egypt
19,NaN,Egypt
26,NaN,Egypt
39,NaN,Egypt
...,...,...
581,NaN,Ansar Allah
587,NaN,Al-Shabaab
590,NaN,Separatists
591,NaN,Libyan Dawn


In [10]:
dfExtraStateWarEntities

,ccode1,SideA,ccode2,SideB
0,210.0,Netherlands,NaN,NaN
1,200.0,United Kingdom,NaN,Algeria
2,640.0,Ottoman Empire,NaN,Saudi Wahhabis
3,230.0,Spain,NaN,San Martin revolutionaries
4,230.0,Spain,NaN,New Granada
...,...,...,...,...
193,290.0,Poland,NaN,NaN
194,325.0,Italy,NaN,NaN
195,369.0,Ukraine,NaN,NaN
196,645.0,Iraq,NaN,NaN


In [11]:
dfESW_POL = pd.concat([dfExtraStateWarEntities[["ccode1", "SideA"]]
                           .rename(columns={"ccode1":"id", "SideA":"name"}), 
                       dfExtraStateWarEntities[["ccode2", "SideB"]]
                           .rename(columns={"ccode2":"id", "SideB":"name"})])\
                .dropna(how="all")
dfESW_POL = dfESW_POL[dfESW_POL["id"].isna()]
dfESW_POL

,id,name
1,NaN,Algeria
2,NaN,Saudi Wahhabis
3,NaN,San Martin revolutionaries
4,NaN,New Granada
5,NaN,Mina Expedition
...,...,...
178,NaN,PKK in Iraq
179,NaN,PKK in Iraq
180,NaN,Palestinians
186,NaN,al-Qaeda & Taliban


In [12]:
dfnonstate_POL = pd.concat([dfNSW_POL, dfISW_POL, dfESW_POL])
dfnonstate_POL["name"] = dfnonstate_POL["name"].str.strip()
dfnonstate_POL = dfnonstate_POL.drop_duplicates()
dfnonstate_POL["id"] = np.arange(10000, 10000+dfnonstate_POL.shape[0])
dfnonstate_POL["type"] = "Non-state actor"
dfnonstate_POL

,name,id,type
0,Te Rauparaha's Ngati Toa,10000,Non-state actor
1,Shaka Zulu,10001,Non-state actor
2,Burma,10002,Non-state actor
3,Buenos Aires,10003,Non-state actor
4,Hongi Hika's Nga Phuhi,10004,Non-state actor
...,...,...,...
171,Polisario,10575,Non-state actor
176,Muhajadin,10576,Non-state actor
178,PKK in Iraq,10577,Non-state actor
186,al-Qaeda & Taliban,10578,Non-state actor


In [13]:
dfPolity = pd.concat([dfStates_POL, dfIOrows, dfnonstate_POL])
dfPolity

,id,name,type,abbr
0,2,United States of America,State,USA
1,20,Canada,State,CAN
2,31,Bahamas,State,BHM
3,40,Cuba,State,CUB
5,41,Haiti,State,HAI
...,...,...,...,...
171,10575,Polisario,Non-state actor,NaN
176,10576,Muhajadin,Non-state actor,NaN
178,10577,PKK in Iraq,Non-state actor,NaN
186,10578,al-Qaeda & Taliban,Non-state actor,NaN


### Verify constraints

- id is unique (b/c primary key)

In [14]:
dfPolity.duplicated(subset=['id']).sum()

0

In [15]:
dfPolity.to_csv(processed_data_path+"polity.csv", index=False)

## Create "polity_dates" table

Table creation statement

```
class Polity_Dates(Base):
    __tablename__ = "polity_dates"

    polity = Column(Integer(5), primary_key=True)
    start_date = Column(Date, primary_key=True)
    end_date = Column(Date)

    __table_args__ = (ForeignKeyConstraint(["polity"], ["polity.id"]),)
```

In [16]:
dfStates

,stateabb,ccode,statenme,styear,stmonth,stday,endyear,endmonth,endday,version
0,USA,2,United States of America,1816,1,1,2016,12,31,2016
1,CAN,20,Canada,1920,1,10,2016,12,31,2016
2,BHM,31,Bahamas,1973,7,10,2016,12,31,2016
3,CUB,40,Cuba,1902,5,20,1906,9,25,2016
4,CUB,40,Cuba,1909,1,23,2016,12,31,2016
...,...,...,...,...,...,...,...,...,...,...
238,NAU,970,Nauru,1999,9,14,2016,12,31,2016
239,MSI,983,Marshall Islands,1991,9,17,2016,12,31,2016
240,PAL,986,Palau,1994,12,15,2016,12,31,2016
241,FSM,987,Federated States of Micronesia,1991,9,17,2016,12,31,2016


In [17]:
dfStateDates = dfStates[['ccode', 'styear', 'stmonth', 'stday', 'endyear', 'endmonth', 'endday']].copy().rename(columns={'ccode': 'polity'})

In [18]:
dfStateDates['start_date'] = pd.to_datetime(dict(year=dfStateDates.styear, month=dfStateDates.stmonth, day=dfStateDates.stday))
dfStateDates['end_date'] = pd.to_datetime(dict(year=dfStateDates.endyear, month=dfStateDates.endmonth, day=dfStateDates.endday))

In [19]:
dfStateDates = dfStateDates[['polity', 'start_date', 'end_date']].copy()
dfStateDates

,polity,start_date,end_date
0,2,1816-01-01,2016-12-31
1,20,1920-01-10,2016-12-31
2,31,1973-07-10,2016-12-31
3,40,1902-05-20,1906-09-25
4,40,1909-01-23,2016-12-31
...,...,...,...
238,970,1999-09-14,2016-12-31
239,983,1991-09-17,2016-12-31
240,986,1994-12-15,2016-12-31
241,987,1991-09-17,2016-12-31


### Verify Constraints

- combo of polity + start date is unique (b/c primary key)
- all polity ids are in polity table

In [20]:
dfStateDates.duplicated(subset=['polity', 'start_date']).sum()

0

In [21]:
len(set(dfStateDates['polity']) - set(dfPolity['id']))

0

In [22]:
dfStateDates['start_date'].min(), dfStateDates['start_date'].max()

(Timestamp('1816-01-01 00:00:00'), Timestamp('2011-07-09 00:00:00'))

In [23]:
dfStateDates['end_date'].min(), dfStateDates['end_date'].max()

(Timestamp('1860-03-15 00:00:00'), Timestamp('2016-12-31 00:00:00'))

In [24]:
dfStateDates.to_csv(processed_data_path+"polity_dates.csv", index=False)